In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import malaya_speech
import random
import numpy as np
import malaya
from sklearn.utils import shuffle
from sklearn.utils.random import sample_without_replacement

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [3]:
malaya_speech.force_alignment.transducer.available_transformer()

,Size (MB),Quantized Size (MB),Language
conformer-transducer,120,32.3,[malay]
conformer-transducer-mixed,120,32.3,"[malay, singlish]"
conformer-transducer-singlish,120,32.3,[singlish]


In [4]:
model = malaya_speech.force_alignment.transducer.transformer(model = 'conformer-transducer', device = 'gpu:0')
singlish_model = malaya_speech.force_alignment.transducer.transformer(model = 'conformer-transducer-singlish', device = 'gpu:0')

2023-03-14 00:14:24.154198: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 00:14:24.193655: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 00:14:24.196036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 00:14:24.196846: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [5]:
asr = malaya_speech.stt.transducer.pt_transformer(model = 'mesolitica/conformer-medium-mixed')
_ = asr.cuda()
_ = asr.eval()

In [6]:
lm = malaya.language_model.kenlm(model = 'bahasa-wiki-news-iium-stt')

In [7]:
lm.score('tak suka hg')

-13.193544387817383

In [8]:
import json
from glob import glob
from tqdm import tqdm

with open('/home/husein/ssd1/speech-bahasa/malay-asr-train.json') as fopen:
    ms = json.load(fopen)

In [9]:
len(ms['X'])

1635599

In [10]:
with open('/home/husein/malaya-speech/singlish-stt-train.json') as fopen:
    sg = json.load(fopen)

In [11]:
len(sg['X'])

3284901

In [12]:
sg['X'][:10]

['/home/husein/ssd2/imda/wav/5-75-tfrecord-1409.wav',
 '/home/husein/ssd2/imda/wav/5-118-tfrecord-1786.wav',
 '/home/husein/ssd2/imda/wav/4-77-tfrecord-2083.wav',
 '/home/husein/ssd2/imda/wav/2-39-tfrecord-78.wav',
 '/home/husein/ssd2/imda/wav/7-63-tfrecord-730.wav',
 '/home/husein/ssd2/imda/wav/5-68-tfrecord-1531.wav',
 '/home/husein/ssd2/imda/wav/0-17-tfrecord-3636.wav',
 '/home/husein/ssd2/imda/wav/1-113-tfrecord-1496.wav',
 '/home/husein/ssd2/imda/wav/7-108-tfrecord-1945.wav',
 '/home/husein/ssd2/imda/wav/5-63-tfrecord-3858.wav']

In [13]:
from datasets import Audio

sr = 16000
minimum = int(0.3 * sr)
audio = Audio(sampling_rate=sr)

In [14]:
def groupby(alignment, length, min_threshold = 0.3):
    r = []
    g = []
    for no, row in enumerate(alignment):
        
        if no > 0 and len(r) and alignment[no]['start'] - alignment[no-1]['end'] >= min_threshold:
            g.append(r)
            r = []
        
        r.append(row)

    return g

In [15]:
directory = 'augmentation-switchboard-v9'
!mkdir {directory}

In [16]:
import torchaudio
import torch
from itertools import permutations

In [17]:
def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

In [ ]:
data = {}
for O in tqdm(range(2000000)):
    try:
        i_ms = sample_without_replacement(len(ms['X']), 3)
        i_sg = sample_without_replacement(len(sg['X']), 3)

        groups = []
        for i in i_ms:
            x = ms['X'][i]
            v = ms['Y'][i]
            y = audio.decode_example(audio.encode_example(x))['array']
            asr_ = asr.forward([y])[0]
            
            if asr_[0][1][0] < 0.07 and calculate_wer(v, asr_[0][0]) > 0.1:
                # print(f'skip ms {i}', v, asr_[0][0], asr_[0][1][0])
                continue
                
            result = model.predict(y, v)

            if ' '.join([r['text'] for r in result['words_alignment']]) != v:
                continue

            grouped = groupby(result['words_alignment'], len(y) / sr)
            # print(grouped)
            for g in grouped:
                y_ = y[int(g[0]['start'] * sr): int(g[-1]['end'] * sr) + minimum]
                if len(y_):
                    v_ = [g_['text'] for g_ in g]
                    groups.append((y_, v_))

        for i in i_sg:
            x = sg['X'][i]
            v = sg['Y'][i]
            y = audio.decode_example(audio.encode_example(x))['array']
            asr_ = asr.forward([y])[0]
            
            if asr_[0][1][0] < 0.1 and calculate_wer(v, asr_[0][0]) > 0.1:
                # print(f'skip sg {i}', v, asr_[0][0], asr_[0][1][0])
                continue
                
            result = singlish_model.predict(y, v)

            if ' '.join([r['text'] for r in result['words_alignment']]) != v:
                continue

            grouped = groupby(result['words_alignment'], len(y) / sr)
            for g in grouped:
                y_ = y[int(g[0]['start'] * sr): int(g[-1]['end'] * sr) + minimum]
                if len(y_):
                    v_ = [g_['text'] for g_ in g]
                    groups.append((y_, v_))
                    
        if not len(groups):
            continue
            
        groups = sorted(groups, key = lambda x: len(' '.join(x[1])), reverse = True)[:7]
        
        strings = [' '.join(g[1]) for g in groups]
        perm = list(permutations(range(len(groups))))
        
        scores = []
        for p in perm:
            s = ' '.join([strings[p_] for p_ in p])
            scores.append(lm.score(s))
            
        best_perm = perm[np.argmax(scores)]
        
        l = 0
        combine_y, combine_v = [], []
        index = 0
        while l < 15 and index < len(groups):
            g = groups[best_perm[index]]
            l_ = len(g[0]) / sr
            if l_ < 1.0:
                index += 1
                continue
            l += l_
            combine_y.append(g[0] / np.abs(g[0]).max())
            combine_v.extend(g[1])
            index += 1

        if len(combine_v):
            audio_path = f'{directory}/{O}.mp3'
            torchaudio.save(audio_path, 
                            torch.tensor(np.concatenate(combine_y).astype('float32')).unsqueeze(0), 
                            sr, format='mp3')
            data[O] = ' '.join(combine_v)
            
    except Exception as e:
        print(e)

  0%|                             | 3339/2000000 [1:55:58<1179:00:31,  2.13s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  1%|▏                           | 16873/2000000 [9:43:37<1084:55:39,  1.97s/it]

2 root error(s) found.
  (0) Invalid argument: Integer division by zero
	 [[node import/floordiv_1 (defined at home/husein/.local/lib/python3.8/site-packages/malaya_boilerplate/frozen_graph.py:384) ]]
	 [[import/non_blank_transcript/_691]]
  (1) Invalid argument: Integer division by zero
	 [[node import/floordiv_1 (defined at home/husein/.local/lib/python3.8/site-packages/malaya_boilerplate/frozen_graph.py:384) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'import/floordiv_1':
  File "usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "home/husein/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "home/husein/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "home/hus

  1%|▎                          | 21311/2000000 [12:17:34<1182:24:16,  2.15s/it]

In [19]:
len(data)

62

In [20]:
data

{0: 'so itulah dia tadi guys',
 1: 'dua minggu dilaporkan',
 2: 'bilakah lantikan ini akan diselesaikan singapore has to firmly protect its religious religiosity in the region at a time of rising',
 3: 'hari hari cuba tun he plans to sleep in a lorry agama islam',
 5: 'so i think shes doing a and i hope that you would help me to compliment that particular tour guide and maybe merangkumi tidak meliputi di dalam tajuk yang mereka',
 6: 'appears to take a harder spinach with triple egg with companies memperhatikan suzann victor',
 7: 'campaigns have to show singaporeans how their action',
 8: 'dont say mine is better do you find that these incentives are truly helpful',
 9: 'what favourite hawker raya sudah ditutup kepada semua lalulintas jaminan bahawa proses pilihan raya yang kendalikan oleh badan berkenaan adalah bersih telus',
 10: 'royal sporting house floridas natural',
 11: 'apa cita cita dia dua tiga kucing belari',
 13: 'the crowd guffawed at comedians the government is steadily 

In [22]:
import IPython.display as ipd
ipd.Audio(f'{directory}/2.mp3')

In [21]:
directory

'augmentation-switchboard-v8'

In [23]:
with open('augment-switchboard-v8.json', 'w') as fopen:
    json.dump(data, fopen)